In [1]:
pip install fastapi


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install "uvicorn[standard]"

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pydantic

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bcrypt

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install python-multipart

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
from fastapi import Depends, FastAPI, HTTPException
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
import uvicorn
from senha import API_KEY
import requests
import json
from models import Usuario, UsuarioBasico, Token, Historia
from db.db import get_database_connection
from services import autenticar_usuario, registrar_usuario, retornar_usuario_por_id, retornar_usuarios, atualizar_usuario, deletar_usuario, registrar_historia, retornar_historias, retornar_historia_por_id, atualizar_historia, deletar_historia

app = FastAPI()

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

mydb = get_database_connection()

@app.post("/registrar-usuario")
def criar_usuario(usuario: Usuario):
    return registrar_usuario(usuario)

@app.get("/usuarios/{id_usuario}")
def retornar_usuario(id_usuario: int):
    return retornar_usuario_por_id(id_usuario)

@app.get("/usuarios")
def ver_todos_usuarios():
    return retornar_usuarios()

@app.put("/usuarios/{id_usuario}")
def atualizar_usuario_por_id(id_usuario: int, usuario: UsuarioBasico):
    return atualizar_usuario(id_usuario, usuario)

@app.delete("/usuarios/{id_usuario}")
def deletar_usuario_por_id(id_usuario: int):
    return deletar_usuario(id_usuario)

# Endpoint para gerar token de acesso
@app.post("/token", response_model=Token)
async def login_for_access_token(form_data: OAuth2PasswordRequestForm = Depends()):
    print("bla")
    usuario = autenticar_usuario(form_data.username, form_data.password)
    if not usuario:
        raise HTTPException(status_code=400, detail="Usuário ou senha incorretos")
    return {"access_token": str(usuario[0]), "token_type": "bearer"}

@app.post("/login", response_model=Token)
async def login(form_data: OAuth2PasswordRequestForm = Depends()):
    usuario = autenticar_usuario(form_data.username, form_data.password)
    if not usuario:
        raise HTTPException(status_code=400, detail="Usuário ou senha incorretos")
    
    # Internamente, chama a rota /token para obter o token de acesso
    access_token = await login_for_access_token(form_data)

    return access_token


headers = { "Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json" }

link = "https://api.openai.com/v1/chat/completions"

id_modelo = "gpt-3.5-turbo"

@app.post("/criar-historia")
async def gerar_historia(historia: Historia, token: str = Depends(oauth2_scheme)):
    body_mensagem = {
    "model" : id_modelo,
    "messages" : [{"role" : "user", "content" : f"Em um mundo onde a demanda por narrativas originais e envolventes é alta, surge uma tecnologia inovadora: uma API capaz de gerar histórias interativas e personalizadas. Esta API oferece oportunidades ilimitadas para explorar novos cenários, personagens e enredos. Pode ser usada por websites, aplicativos, equipes de escritores, desenvolvedores de jogos e até mesmo por educadores para ensinar elementos de enredo, estrutura de história e desenvolvimento de personagem de forma interativa. Essa ferramenta não apenas oferece diversidade e interatividade, mas também se adapta às necessidades dos usuários: Inspiração para criações individuais. Testes de diferentes abordagens para histórias. Narrativas personalizadas que se adaptam às escolhas dos leitores.Com base neste contexto, por favor, continue a história a partir deste ponto... {historia.descricao}"}],
}

    response = requests.post(link, headers=headers, data=json.dumps(body_mensagem))

    registrar_historia(historia, response.json()["choices"][0]["message"]["content"])
    
    return response.json()["choices"][0]["message"]["content"]


@app.get("/historias")
def ver_todas_historias():
    return retornar_historias()

@app.get("/historias/{id_historia}")
def retornar_historia(id_historia: int):
    return retornar_historia_por_id(id_historia)

@app.put("/historias/{id_historia}")
def atualizar_historia_por_id(id_historia: int, historia: Historia):
    return atualizar_historia(id_historia, historia)

@app.delete("/historias/{id_historia}")
def deletar_historia_por_id(id_historia: int):
    return deletar_historia(id_historia)


if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [24782]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56144 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56144 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56148 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56158 - "GET /openapi.json HTTP/1.1" 200 OK


## TESTES

In [ ]:
import requests

class TestAPI():
    base_url = "http://127.0.0.1:8000"

    def test_hello_endpoint(self):
        response = requests.get(f"{self.base_url}/usuarios")
        
        if response.status_code == 200:
            print("Teste de listagem de usuários: OK")
        else:
            print("Teste de listagem de usuários: Falhou")



test = TestAPI()
test.test_hello_endpoint()
    

# Executar os testes



ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /usuarios (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11741c990>: Failed to establish a new connection: [Errno 61] Connection refused'))